# RNN Example

Import neccesary packages for tensorflow

In [184]:
import tensorflow as tf
from tensorflow.python.ops import rnn_cell

In [185]:
# Training Parameters
learning_rate = 0.0001 #Play with learning rate and repor results
training_steps = 2000
batch_size = 32
display_step=200

In [186]:
#Load mnist
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [187]:
# Network Parameters
num_input = 28 # MNIST data input (img shape: 28*28)
timesteps = 28 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 10 # MNIST total classes (0-9 digits)

In [188]:

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

### Define RNN cell:

A) LSTM 

B) GRU

C) Multi Layer

D) Multi Layer with Dropout

## SIMPLE LSTM

In [178]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0, reuse = tf.AUTO_REUSE)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']


logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)


In [179]:
# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

In [180]:
# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [181]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")
    acc_total=0.0
    for step in range(0, mnist.test.images.shape[0]/batch_size):
        #Get batches from the mnist.test.images and mnist.test.labels
        acc=sess.run(accuracy, feed_dict={X: batch_x, Y: batch_y})
        acc_total+=acc
    print("Testing Accuracy:",acc_total/(mnist.test.images.shape[0]/batch_size))

Step 1, Minibatch Loss= 2.4785, Training Accuracy= 0.094
Step 200, Minibatch Loss= 2.4323, Training Accuracy= 0.312
Step 400, Minibatch Loss= 2.3907, Training Accuracy= 0.250
Step 600, Minibatch Loss= 2.3093, Training Accuracy= 0.156
Step 800, Minibatch Loss= 2.3608, Training Accuracy= 0.156
Step 1000, Minibatch Loss= 2.3351, Training Accuracy= 0.250
Step 1200, Minibatch Loss= 2.3043, Training Accuracy= 0.094
Step 1400, Minibatch Loss= 2.1217, Training Accuracy= 0.188
Step 1600, Minibatch Loss= 2.1726, Training Accuracy= 0.312
Step 1800, Minibatch Loss= 2.2053, Training Accuracy= 0.250
Step 2000, Minibatch Loss= 2.1985, Training Accuracy= 0.219
Optimization Finished!
('Testing Accuracy:', 0.21875)


## MULTI LAYER LSTM

In [189]:
num_layers =  3
def lstm_cell(dropout = 0.5):
    cell = tf.contrib.rnn.BasicLSTMCell(num_hidden, reuse= tf.AUTO_REUSE)
    return  tf.contrib.rnn.DropoutWrapper( cell, output_keep_prob=1.0 - dropout)
stacked_lstm = tf.contrib.rnn.MultiRNNCell(
    [lstm_cell(0.4) for _ in range(num_layers)])
initial_state = state = stacked_lstm.zero_state(batch_size, tf.float32)


In [190]:

for i in range(timesteps):
    input_rnn = X[:,i,:]
    output, state = stacked_lstm(input_rnn, state)

final_state = state
final_output = output
final_output


<tf.Tensor 'multi_rnn_cell_305/cell_2_27/dropout/mul:0' shape=(32, 128) dtype=float32>

In [191]:
logits = tf.matmul(final_output, weights['out']) + biases['out']
prediction = tf.nn.softmax(logits)

### Compute logits with w1 and b1 -- Multi Layer

In [192]:
# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

In [193]:
# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [194]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")
    acc_total=0.0
    for step in range(0, mnist.test.images.shape[0]/batch_size):
        #Get batches from the mnist.test.images and mnist.test.labels
        acc=sess.run(accuracy, feed_dict={X: batch_x, Y: batch_y})
        acc_total+=acc
    print("Testing Accuracy:",acc_total/(mnist.test.images.shape[0]/batch_size))

Step 1, Minibatch Loss= 2.9711, Training Accuracy= 0.062
Step 200, Minibatch Loss= 2.3199, Training Accuracy= 0.125
Step 400, Minibatch Loss= 2.4173, Training Accuracy= 0.031
Step 600, Minibatch Loss= 2.2670, Training Accuracy= 0.094
Step 800, Minibatch Loss= 2.2451, Training Accuracy= 0.219
Step 1000, Minibatch Loss= 2.2162, Training Accuracy= 0.188
Step 1200, Minibatch Loss= 2.2532, Training Accuracy= 0.188
Step 1400, Minibatch Loss= 2.2023, Training Accuracy= 0.219
Step 1600, Minibatch Loss= 2.0980, Training Accuracy= 0.250
Step 1800, Minibatch Loss= 2.1850, Training Accuracy= 0.250
Step 2000, Minibatch Loss= 2.2942, Training Accuracy= 0.188
Optimization Finished!
('Testing Accuracy:', 0.1773838141025641)
